In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
# from simpletransformers.classification import ClassificationModel
# from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import ast
import numpy as np
import re

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

from google.colab import files
uploaded = files.upload()

Saving multilabel_classification to multilabel_classification


In [45]:
multi_label_classification = pd.read_csv("/content/multilabel_classification", usecols=["text", "labels"])
multi_label_classification.head()

,text,labels
0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...


In [0]:
# summary_state = summary_state.drop(columns=["State", "Count"])
# summary_state = summary_state.rename(columns={"Summary":"text", "Label":"labels"})
# summary_state = summary_state.reset_index(drop=True)
multi_label_classification = multi_label_classification.reset_index()

In [47]:
separation = int(0.8 * len(multi_label_classification))
train_data = multi_label_classification[:separation]
test_data = multi_label_classification[separation:]
print(len(test_data))
train_data

32


,index,text,labels
0,0,The districts of Guidan-Roumji in Niger and Do...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. ...
1,1,The week 6 was marked by a sudden apparition o...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. ...
2,2,Spatial analysis shows an epidemic area common...,[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. ...
3,3,During week 14 it is noted that: Burkina Faso:...,[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. ...
4,4,Burkina Faso: Titao was the last district in a...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. ...
...,...,...,...
123,123,From week 40 to 43 (corresponding to October 2...,[0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. ...
124,124,From week 44 to 47 (corresponding to November ...,[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. ...
125,125,From week 48 to 52 (corresponding to December ...,[0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. ...
126,126,The first week of the year 2017 is marked by t...,[0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. ...


In [7]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ar9iknld
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ar9iknld
     |████████████████████████████████| 3.8MB 3.4MB/s 
     |████████████████████████████████| 1.1MB 24.1MB/s 
     |████████████████████████████████| 890kB 15.6MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=675545 sha256=b5af6e2ec8aa26c1f08e5bb6067aee6ae005190a7b3a51dc0eff00ea55e5837e
  Stored in directory: /tmp/pip-ephem-wheel-cache-cjwyb2r0/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ad90d5419a0b628701d79fd1dedd1c9567d647acf33f58ba5a672e3e48240787
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [0]:
device = torch.device("cuda")
from transformers import LongformerModel, LongformerTokenizer, LongformerConfig

In [9]:
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
model.cuda()

INFO:filelock:Lock 140073655189232 acquired on /root/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


INFO:filelock:Lock 140073655189232 released on /root/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
INFO:filelock:Lock 140076388762176 acquired on /root/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


INFO:filelock:Lock 140076388762176 released on /root/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


INFO:filelock:Lock 140073629560904 acquired on /root/.cache/torch/transformers/ea6da73c9ecf0c07b29510e423dfd58f1b8b59cba3c89383189ec8b0b6d00e82.4859554090880d98c2ceab35aa57afd197958c9c3b161bbfe07aeefb84557e54.lock


INFO:filelock:Lock 140073629560904 released on /root/.cache/torch/transformers/ea6da73c9ecf0c07b29510e423dfd58f1b8b59cba3c89383189ec8b0b6d00e82.4859554090880d98c2ceab35aa57afd197958c9c3b161bbfe07aeefb84557e54.lock
INFO:filelock:Lock 140073632231608 acquired on /root/.cache/torch/transformers/dfc92dbbf5c555abf807425ebdb22b55de7a17e21fe1c48cbaa5764982c1d9c0.cd65234711d2e83d420aa696eb9186cdec6ab79ef8bf090b442cf249443dfa92.lock


INFO:filelock:Lock 140073632231608 released on /root/.cache/torch/transformers/dfc92dbbf5c555abf807425ebdb22b55de7a17e21fe1c48cbaa5764982c1d9c0.cd65234711d2e83d420aa696eb9186cdec6ab79ef8bf090b442cf249443dfa92.lock


LongformerModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): BertSelfOutpu

In [10]:
torch.cuda.empty_cache()
#note when encoded the start of every token is added with an <s> for classification purposes
input_ids = tokenizer.encode("Hello, my dog is cute")  # Batch size 1
print(tokenizer.decode(input_ids))
print(tokenizer.decode(input_ids[0]))

<s> Hello, my dog is cute</s>
<s>


In [0]:
tokenized = multi_label_classification["text"]\
                                    .apply(lambda x: tokenizer.encode(x, add_special_tokens_tokens=True))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)
#padded according to max_length so we can feed the model in batches - padding works as such, add 0s after the length
#of the original sequence l such that the new length is = max_len = 2530
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


2530


In [0]:
np.array(padded.shape)
#Add masking so that know BERT knows where to attend to => so it can ignore padding
#adds a 1 everwhere where the condition of padded != 0 is satisfied, in other words everywhere there isn't padding
attention_mask = np.where(padded != 0, 1, 0)
#add global attention represented with 2 - for now just add 1 at the classification <s> token
attention_mask[:, [0, -1]] = 2
print(attention_mask)
device = torch.device("cuda")

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)

def create_save_feature_representation():
  first_time = True
  for i in range(0,len(input_ids),2):
    with torch.no_grad():
      predictions, hidden_states = model(input_ids[i:i+2], attention_mask=attention_mask[i:i+2])
      scores = predictions[:,0,:].cpu().numpy()
      if not first_time:
        curr = np.load("lang_features.npy")
        scores = np.concatenate((curr, scores))
      else:
        first_time = False
      np.save("lang_features.npy",scores)

# create_save_feature_representation()

In [0]:
features = np.load("lang_features.npy")
#convert the labels into numpy arrays since they are stored as string representations
multi_label_classification["labels"] = multi_label_classification["labels"].apply(lambda x: np.fromstring(x[1:-1], dtype=np.float32, sep=' '))

In [43]:
train_features, test_features = features[:separation], features[separation:]
hidden_size = features.shape[1]

#convert to float datatype that can be then cast to a tensor in PyTorch
labels = np.array([label for label in np.array(multi_label_classification["labels"].values)])
train_labels, test_labels = labels[:separation], labels[separation:]
multi_label_classification.iloc[0]["labels"].shape

(20,)

In [35]:
print(train_labels.dtype)
epochs = 50
neural_network = nn.Sequential(
                    nn.Linear(hidden_size, 1000),
                    nn.ReLU(),
                    nn.BatchNorm1d(1000),
                    nn.Linear(1000, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 600),
                    nn.ReLU(),
                    nn.BatchNorm1d(600),
                    nn.Linear(600, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 200),
                    nn.ReLU(),
                    nn.BatchNorm1d(200),
                    nn.Linear(200, 20),
                    nn.Sigmoid(),
                    )
neural_network.cuda()
dtype = torch.cuda.FloatTensor
neural_network.type = torch.cuda.FloatTensor
train_features, test_features = torch.tensor(train_features).to(device), torch.tensor(test_features).to(device)
train_labels, test_labels = torch.tensor(train_labels).to(device), torch.tensor(test_labels).to(device)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(neural_network.parameters(), lr=0.0001)

for epoch in range(epochs):
  optimizer.zero_grad()
  output = neural_network(train_features)
  loss = criterion(output, train_labels)
  print("Epoch {0} with training loss: {1}".format(epoch, loss))
  loss.backward()
  optimizer.step()    

float32
Epoch 0 with training loss: 0.7511270642280579
Epoch 1 with training loss: 0.7167929410934448
Epoch 2 with training loss: 0.6995739936828613
Epoch 3 with training loss: 0.6877074241638184
Epoch 4 with training loss: 0.6787651777267456
Epoch 5 with training loss: 0.6714318990707397
Epoch 6 with training loss: 0.6654398441314697
Epoch 7 with training loss: 0.6603212356567383
Epoch 8 with training loss: 0.6558641195297241
Epoch 9 with training loss: 0.6518908739089966
Epoch 10 with training loss: 0.6484514474868774
Epoch 11 with training loss: 0.6455328464508057
Epoch 12 with training loss: 0.6429403424263
Epoch 13 with training loss: 0.6405662894248962
Epoch 14 with training loss: 0.6384145021438599
Epoch 15 with training loss: 0.6364120244979858
Epoch 16 with training loss: 0.6345706582069397
Epoch 17 with training loss: 0.6328554153442383
Epoch 18 with training loss: 0.6312694549560547
Epoch 19 with training loss: 0.629805862903595
Epoch 20 with training loss: 0.628421068191528

In [36]:
def convert_to_thresholds(data):
  return data>0.5

def test_model():
  preds = neural_network(test_features).cpu().data.numpy()
  correct = test_labels.cpu().numpy()
  print(correct)
  preds = convert_to_thresholds(preds)
  print("F1 Score: ", f1_score(preds, correct, average="micro"))
  print("ROC-AUC: ", roc_auc_score(correct, preds))

test_model()

[[0. 1. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 1. 1. 0. 1. 1. 1. 0.]
 [1. 0. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 0. 1. 0.]
 [0. 1. 1. 1. 1. 0. 1. 0.]
 [0. 1. 1. 0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 1. 0. 1.]
 [0. 1. 1. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 1. 1.]]
F1 Score:  0.5588235294117647
ROC-AUC:  0.5018744829311744


##Tests on simplified 2 state binary classification problem
**Initial results tested with 20 labels:**
F1 Score: 0.53
ROC-AUC: 0.51

**Tested on only the most present 8 labels** - as done with the forecasting pipeline (to check if this is a signal to noise ratio)
F1 Score: 0.558
ROC-AUC: 0.501

So the issue here isn't due to noise - the Longformer is just not able to learn a meaningful enough feature representation that can map to the labels.

Let's try to finetune our Longformer model first on some domain-specific text - reports from the WHO that outline how they plan to defeat Meningitis by 2030. After finetuning on this data, let's try to regenerate our feature representations of our report text and see if that changes our results with the multilabel classifier

In [24]:
#As input to finetune model - need to tokenize and feed in sentences into longformer
#In order to feed text into the longformer - need to pass in 3 things:
#1. input_ids which are the tokenized sentences (with the correct tags e.g. <s>)
#2. input_masks telling model where to pay attention to - i.e. where to ignore padding etc.
#3. token type ids which tell models about relationship of different sentences (e.g. relevant to question and answering datasets) 
# since our input consists of single sentences, we don't need it however
def tokenize_sentences(sentences):
  for sentence in sentences:
    #encodings is a dictionary with the encoded sequence and additional information like the attention mask
    enocdings = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=4096, pad_to_max_length=True, return_attention_mask=True, return_token_type_ids=True)
    return encodings

  return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32')


with open("finetune.txt", "r") as f:
  corpus = f.read().strip()
  corpus = corpus.replace('."', '".')
  sentences = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", corpus)
  max_s = 0
  s_t = ""
  for s in sentences:
    if len(s)>max_s:
      max_s = len(s)
      s_t = s
  print(max_s)
  #save the last 2000 characters for evaluation as the test test
  train_sentences, test_sentences = sentences[:-2000], sentences[-2000:]

input_ids, input_masks = tokenize_sentences(sentences)

# config = LongformerConfig(dropout=0.2, attention_dropout=0.2)
# config.output_hidden_states = False
#note we're gonna be using the Longformer model we defined above so we don't need to define a new model here



1157
[[0, 20, 529, 21, 18783, 30, 925, 7157, 493, 12024, 428, 5654, 338, 36, 23007, 6073, 19885, 6, 27610, 833, 30331, 868, 34477, 8, 29054, 1938, 6, 13387, 4722, 1387, 13, 5, 3877, 11965, 6, 14794, 43, 8, 6020, 2995, 24048, 36, 23122, 835, 9, 8851, 41866, 8, 18683, 8029, 6, 928, 6, 315, 5752, 9, 2860, 1444, 8, 2874, 2487, 43, 36, 7048, 40396, 112, 13, 5, 4026, 8, 40396, 132, 13, 5, 889, 9, 3597, 322, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1